In [1]:
#installing library

import pandas as pd 
from pandas import Series, DataFrame
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient

In [2]:
#uploading data dictionary

data_dict = pd.read_excel("NYPD_Arrest_YTD_DataDictionary.xlsx", sheet_name='Column Info')
data_dict

,Data Dictionary - Column Information,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Column Name,Column Description,"Term, Acronym, or Code Definitions","Additional Notes \n(where applicable, include ..."
1,ARREST_KEY,Randomly generated persistent ID for each arrest,NaN,NaN
2,ARREST_DATE,Exact date of arrest for the reported event,NaN,NaN
3,PD_CD,Three digit internal classification code (more...,NaN,NaN
4,PD_DESC,Description of internal classification corresp...,NaN,NaN
5,KY_CD,Three digit internal classification code (more...,NaN,NaN
6,OFNS_DESC,Description of internal classification corresp...,NaN,NaN
7,LAW_CODE,Law code charges corresponding to the NYS Pena...,NaN,NaN
8,LAW_CAT_CD,"Level of offense: felony, misdemeanor, violation",NaN,NaN
9,ARREST_BORO,"Borough of arrest. B(Bronx), S(Staten Island),...",NaN,NaN


In [3]:
#source 1 

df_raw = pd.read_csv("NYPD_Arrest_Data__Year_to_Date__20240229.csv")
print(df_raw.info())
print(df_raw.shape)
print(df_raw.columns)
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226872 entries, 0 to 226871
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ARREST_KEY                226872 non-null  int64  
 1   ARREST_DATE               226872 non-null  object 
 2   PD_CD                     226870 non-null  float64
 3   PD_DESC                   226872 non-null  object 
 4   KY_CD                     226855 non-null  float64
 5   OFNS_DESC                 226872 non-null  object 
 6   LAW_CODE                  226872 non-null  object 
 7   LAW_CAT_CD                225273 non-null  object 
 8   ARREST_BORO               226872 non-null  object 
 9   ARREST_PRECINCT           226872 non-null  int64  
 10  JURISDICTION_CODE         226872 non-null  int64  
 11  AGE_GROUP                 226872 non-null  object 
 12  PERP_SEX                  226872 non-null  object 
 13  PERP_RACE                 226872 non-null  o

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,261265483,01/03/2023,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,B,49,0,18-24,M,BLACK,1027430,251104,40.855793,-73.843908,POINT (-73.843908 40.855793)
1,261271301,01/03/2023,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,S,120,0,25-44,M,WHITE,962808,174275,40.644996,-74.077263,POINT (-74.077263 40.644996)
2,261336449,01/04/2023,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,K,61,0,<18,M,BLACK,995118,155708,40.594054,-73.960866,POINT (-73.960866 40.594054)
3,261328047,01/04/2023,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,Q,114,0,18-24,M,BLACK,1007694,219656,40.769552,-73.915361,POINT (-73.915361 40.769552)
4,261417496,01/05/2023,244.0,"BURGLARY,UNCLASSIFIED,UNKNOWN",107.0,BURGLARY,PL 1402000,F,B,44,0,25-44,F,BLACK,1007174,239542,40.824135,-73.917170,POINT (-73.91717 40.824135)


In [4]:
#source 2 

response = requests.get("https://data.cityofnewyork.us/resource/uip8-fykc.json")
response.status_code

200

In [5]:
data = response.json()

In [6]:
api_df = pd.DataFrame(data)
print(api_df.info())
print(api_df.shape)
print(api_df.columns)
api_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   arrest_key                   1000 non-null   object
 1   arrest_date                  1000 non-null   object
 2   pd_cd                        1000 non-null   object
 3   pd_desc                      1000 non-null   object
 4   ky_cd                        1000 non-null   object
 5   ofns_desc                    1000 non-null   object
 6   law_code                     1000 non-null   object
 7   law_cat_cd                   992 non-null    object
 8   arrest_boro                  1000 non-null   object
 9   arrest_precinct              1000 non-null   object
 10  jurisdiction_code            1000 non-null   object
 11  age_group                    1000 non-null   object
 12  perp_sex                     1000 non-null   object
 13  perp_race                    1000 

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,x_coord_cd,y_coord_cd,latitude,longitude,geocoded_column,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_efsh_h5xi
0,261265483,2023-01-03T00:00:00.000,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1600500,F,B,49,...,1027430,251104,40.855793,-73.843908,"{'type': 'Point', 'coordinates': [-73.843908, ...",59,5,12,32,11270
1,261271301,2023-01-03T00:00:00.000,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,S,120,...,962808,174275,40.644996,-74.077263,"{'type': 'Point', 'coordinates': [-74.077263, ...",4,1,13,74,10369
2,261336449,2023-01-04T00:00:00.000,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1601001,F,K,61,...,995118,155708,40.594054,-73.960866,"{'type': 'Point', 'coordinates': [-73.960866, ...",32,2,15,36,18183
3,261328047,2023-01-04T00:00:00.000,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,Q,114,...,1007694,219656,40.769552,-73.915361,"{'type': 'Point', 'coordinates': [-73.915361, ...",39,3,4,72,16860
4,261417496,2023-01-05T00:00:00.000,244,"BURGLARY,UNCLASSIFIED,UNKNOWN",107,BURGLARY,PL 1402000,F,B,44,...,1007174,239542,40.824135,-73.91717,"{'type': 'Point', 'coordinates': [-73.91717, 4...",34,5,43,25,10929


In [7]:
import os

CONNECTION_STRING_AZURE_STORAGE = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

if not CONNECTION_STRING_AZURE_STORAGE:
    raise ValueError("No Azure Storage connection string found")

CONTAINER_AZURE = 'nypd-arrest'
blob_name = "nypd-arrest.csv"

# Convert DataFrame to CSV and get the string
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

# Print confirmation message
print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")



Uploaded nypd-arrest.csv to Azure Blob Storage in container nypd-arrest.
